# Рекомендация тарифов

## Описание проекта
В нашем распоряжении данные о поведении клиентов, которые уже перешли на новые тарифы. Нужно построить модель для задачи классификации, которая выберет подходящий тариф для пользователей архивных тарифов.
## Цель проекта
Построить модель с максимально большим значением accuracy (не менее 0,75).
## Описание данных
Каждый объект в наборе данных — это информация о поведении одного пользователя за месяц. Известно:

`сalls` — количество звонков,

`minutes` — суммарная длительность звонков в минутах,

`messages` — количество sms-сообщений,

`mb_used` — израсходованный интернет-трафик в Мб,

`is_ultra` — каким тарифом пользовался в течение месяца («Ультра» — 1, «Смарт» — 0).

## 1. Загрузка и обзор данных

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.dummy import DummyClassifier

In [3]:
df.head()

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


In [5]:
df.describe()

,calls,minutes,messages,mb_used,is_ultra
count,3214.000000,3214.000000,3214.000000,3214.000000,3214.000000
mean,63.038892,438.208787,38.281269,17207.673836,0.306472
std,33.236368,234.569872,36.148326,7570.968246,0.461100
min,0.000000,0.000000,0.000000,0.000000,0.000000
25%,40.000000,274.575000,9.000000,12491.902500,0.000000
50%,62.000000,430.600000,30.000000,16943.235000,0.000000
75%,82.000000,571.927500,57.000000,21424.700000,1.000000
max,244.000000,1632.060000,224.000000,49745.730000,1.000000


### Вывод
Данные загружены и готовы к построению модели. Пропусков в данных нет. На текущий момент видно, что среди пользователей новых тарифов "Ультра" только у 31 % из них.

## 2. Построение моделей
В исследовании будут испытаны три модели: дерево решений, случайный лес и логистическая регрессия.

### 2.1. Разделение датасета на выборки
Подготовим тренировочную, валидационную и тестовую выборки в соотношении 6:2:2 соответственно

In [6]:
features = df.drop('is_ultra', axis=1)
target = df['is_ultra']

In [7]:
# для разбиения на 3 части используем train_test_split 2 раза последовательно
features_train, x, target_train, y = train_test_split(features, target, test_size=0.4, train_size=0.6, random_state=12345, stratify=target)
features_valid, features_test, target_valid, target_test = train_test_split(x, y, test_size=0.5, train_size=0.5, random_state=12345, stratify=y)

In [8]:
display(features_train.shape)
display(features_valid.shape)
display(features_test.shape)

(1928, 4)

(643, 4)

(643, 4)

### 2.2. Обучение дерева решений
Подготовим и проверим на валидационной выборке дерево решений с различной глубиной

In [9]:
for depth in range(1, 7):
    model = DecisionTreeClassifier(max_depth=depth, random_state=12345)
    model.fit(features_train, target_train)
    predictions = model.predict(features_valid)
    print(f'max_depth = {depth} : {accuracy_score(target_valid, predictions):.2}')

max_depth = 1 : 0.74
max_depth = 2 : 0.77
max_depth = 3 : 0.78
max_depth = 4 : 0.75
max_depth = 5 : 0.79
max_depth = 6 : 0.77


#### Вывод
Лучший результат 0.79 показала модель с глубиной дерева равной 5

### 2.2. Обучение случайного леса
Подготовим и проверим на валидационной выборке случайный лес с различной глубиной деревьев и количеством деревьев

In [10]:
best_result = 0
estimators = 0
best_depth = 0
best_leaf = 0
for est in range(1, 10):
    for depth in range(1, 12):
        for leaf in range(2, 10):
            model = RandomForestClassifier(random_state=12345, n_estimators=est, max_depth=depth, min_samples_leaf=leaf)
            model.fit(features_train, target_train)
            predictions = model.predict(features_valid)
            result = accuracy_score(predictions, target_valid)
            if result > best_result:
                estimators = est
                best_depth = depth
                best_result = result
                best_leaf = leaf

print(f"Accuracy наилучшей модели на валидационной выборке: {best_result:.2}")
print("Глубина деревьев:", best_depth, "\nКоличество деревьев:", estimators, "\nКоличество элементов в листе:", best_leaf)

Accuracy наилучшей модели на валидационной выборке: 0.82
Глубина деревьев: 9 
Количество деревьев: 5 
Количество элементов в листе: 8


#### Вывод
Лучший результат 0.82 показала модель с глубиной деревьев равной 9, количеством деревьев - 5 и минимальным количеством элементов в листе - 8

### 2.3. Обучение модели логистической регрессии
Подготовим и проверим на валидационной выборке модель логистической регрессии

In [11]:
model = LogisticRegression(random_state=12345)
model.fit(features_train, target_train)
predictions = model.predict(features_valid)
print(f'Accuracy = {accuracy_score(predictions, target_valid):.2}')

Accuracy = 0.74


#### Вывод
На валидационной выборке модель имеет худший результат среди рассмотренных

### Вывод
Мы обучили три модели: дерево решений, случайный лес и логистическую регрессию. Среди рассмотренных моделей лучший результат 0,82 показала модель случайного леса с количеством деревьев 5, глубиной 9 и минимальным количеством элеметов 8

## 3. Проверка моделей
Проверим лучшую модель на тестовой выборке

In [12]:
forest_model = RandomForestClassifier(random_state=12345, n_estimators=5, max_depth=9, min_samples_leaf=8)
forest_model.fit(features_train, target_train)

RandomForestClassifier(max_depth=9, min_samples_leaf=8, n_estimators=5,
                       random_state=12345)

In [13]:
forest_predictions = forest_model.predict(features_test)

In [14]:
print(f'Accuracy случайного леса = {accuracy_score(forest_predictions, target_test):.3}')

Accuracy случайного леса = 0.81


### Вывод
На тестовой выборке модель показала точность 0.81, чуть ниже, чем на валидационной.

## 4. Проверка на адекватность
Для проверки на адекватность будем использовать наивный классификатор, который выдает в качестве прогноза наиболее частое значение из выборки.

In [15]:
dummy = DummyClassifier(random_state=12345)
dummy.fit(features_train, target_train)
dummy_predictions = dummy.predict(features_test)
print(f'Accuracy наивного классификатора = {accuracy_score(dummy_predictions, target_test):.3}')

Accuracy наивного классификатора = 0.694


### Вывод
Наивный классификатор угадывает в 69 % случаев.

## Вывод
Таким образом, в исследовании были расмотрены модели дерева решений, случайного леса и логистической регрессии. На валидационной выборке требуемый порог в 0.75 преодалели модель случайного леса и дерево решений. Наилучший результат accuracy=0.82 на валидационной выборке показала модель случайного леса со следующими гипермараметрами: количество деревьев - 5, глубина - 9, минимальное количество элементов в листе - 8. На тестовой выборке она показала точность 0.81. Полученный результат лучше наивной классификации примерно на 22 %.